In [1]:
import os, sys
os.environ["CUDA_VISIBLE_DEVICES"] = "5,"
os.environ["PYOPENGL_PLATFORM"] = "osmesa"
from pathlib import Path
from omegaconf import OmegaConf
import numpy as np
import torch

import smplx
import meshplot as mp
from psbody.mesh import Mesh
from psbody.mesh.colors import name_to_rgb

from flex.pretrained_models.loader_grabnet import Trainer as RHGrasp_Loader
from flex.pretrained_models.loader_vposer import Trainer as VPoser_Loader
from flex.tools.config import Config
from flex.tools.utils import get_ground, replace_topk, aa2rotmat, rotmat2aa
from run import Optimize

device = "cuda:0"

# CUDA_VISIBLE_DEVICES=6, python run.py \
# --obj_name stapler \
# --receptacle_name receptacle_aabb_TvStnd1_Top3_frl_apartment_tvstand \
# --ornt_name all \
# --gender 'female'

In [2]:
obj_name = "stapler"
receptacle_name = "receptacle_aabb_TvStnd1_Top3_frl_apartment_tvstand"
ornt_name = "all"
index = 0
bs = 1

cfg_vp = OmegaConf.structured(Config)                   # Load base config
cfg_yaml = OmegaConf.load('flex/configs/flex.yaml')    # Load yaml config
cfg_rh = OmegaConf.load('flex/configs/rh.yaml')
cfg_vp = OmegaConf.merge(cfg_vp, cfg_yaml)
cfg_rh.batch_size = cfg_vp.bs = bs

sbj_m = smplx.create(
    model_path=cfg_vp.smplx_dir, model_type='smplx', gender='neutral',
    num_pca_comps=24, batch_size=1
)

dset_info_dict = dict(np.load('data/replicagrasp/dset_info.npz', allow_pickle=1))
transl_grab, orient_grab, recept_idx = dset_info_dict[f'{obj_name}_{receptacle_name}_{ornt_name}_{index}']
obj_transl = torch.Tensor(transl_grab).to(device)
obj_global_orient = rotmat2aa(torch.Tensor(orient_grab).reshape(1,1,1,9)).reshape(1,3).to(device)

recept_dict = dict(np.load('data/replicagrasp/receptacles.npz', allow_pickle=1))
obstacles_dict = {receptacle_name: recept_dict[receptacle_name][recept_idx]}
# obstacles_dict = {}

In [3]:
object_mesh = Mesh(filename=os.path.join(cfg_vp.obj_meshes_dir, obj_name + '.ply'), vscale=1.)
obj_verts = np.matmul(object_mesh.v, orient_grab.T) + transl_grab
mp_viewer = mp.plot(obj_verts, object_mesh.f, name_to_rgb['blue'])

mp_viewer.add_mesh(sbj_m().vertices[0].detach().cpu().numpy(), sbj_m.faces, name_to_rgb['green'])

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4612175…

1

In [4]:
tto = Optimize(cfg_rh=cfg_rh, cfg_vp=cfg_vp)
_ = tto.mime_net.eval()
_ = tto.coarse_net.eval()
_ = tto.refine_net.eval()


Total Trainable Parameters for Generative Hand Grasping Model (CoarseNet) is 14.04 M.
Total Trainable Parameters for Generative Hand Grasping Model (RefineNet) is 3.26 M.
------> Loading CoarseNet model (pre-trained on GRAB right-hand grasps) from /data/3D_dataset/hand_grasp/GrabNet/grabnet/models/coarsenet.pt
------> Loading RefineNet model (pre-trained on GRAB right-hand grasps) from /data/3D_dataset/hand_grasp/GrabNet/grabnet/models/refinenet.pt


Total Trainable Parameters for Human Generative Model (VPoser) is 0.94 M.
------> Loading VPoser model (pre-trained on AMASS) from /data/3D_dataset/smpl_related/models/V02_05/snapshots/V02_05_epoch=13_val_loss=0.03.ckpt



In [5]:
object_mesh = Mesh(filename=os.path.join(cfg_vp.obj_meshes_dir, obj_name + '.ply'), vscale=1.)
object_mesh.reset_normals()
object_mesh = [object_mesh.v, object_mesh.f]

obj_bps, _, a_inits = tto.displace(obj_name, [obj_transl, obj_global_orient], 1, object_mesh)
t_inits, g_inits, z_inits, w_inits = tto.get_inits([obj_transl, obj_global_orient], obj_bps, 1)

curr_res = tto.perform_optim(z_inits, t_inits, g_inits, w_inits, a_inits,
                                obstacles_dict, obj_bps, object_mesh,
                                obj_transl, obj_global_orient, obj_name, 'latent')

# (*) Save topk results.
# results = replace_topk(curr_res, min(bs, 10))
# final_result = {
#     'human_vertices': results['human_vertices'][0].detach().cpu().numpy(),
#     'pose': results['pose_final'][0].reshape(21, 3).detach().cpu().numpy(),
#     'transl': results['transl_final'][0].detach().cpu().numpy(),
#     'global_orient': aa2rotmat(results['global_orient_final'])[0].view(3, 3).detach().cpu().numpy(),
#     'rh_verts': results['rh_verts'][0].detach().cpu().numpy(),
#     'z_final': results['z_final'][0].detach().cpu().numpy(),
# }
body_params = {
    'transl': curr_res['transl_final'][0:1],
    'global_orient': curr_res['global_orient_final'][0:1],
    'body_pose': curr_res['pose_final'][0:1]
}
body_vertices = sbj_m(**body_params).vertices.detach().cpu().numpy()[0]
body_vertices[..., 2] -= body_vertices[..., 2].min(-1, keepdims=True)[0]


Total Trainable Parameters for Pose-Ground Generative Model (PGP) is 2.15 K.
------> Loading PGP model (pre-trained on AMASS) from flex/pretrained_models/ckpts/pgp.pth

Total Trainable Parameters for FLEX is 292922.


  0%|          | 0/500 [00:00<?, ?it/s]

Starting TTO loss: 21.53
Ending TTO loss: 0.04777
Best   TTO loss: 0.04257


In [6]:
object_mesh = Mesh(filename=os.path.join(cfg_vp.obj_meshes_dir, obj_name + '.ply'), vscale=1.)
obj_verts = np.matmul(object_mesh.v, orient_grab.T) + transl_grab
mp_viewer = mp.plot(obj_verts, object_mesh.f, name_to_rgb['blue'])

# mp_viewer.add_mesh(
#     obstacles_dict[receptacle_name][0], 
#     obstacles_dict[receptacle_name][1], 
#     name_to_rgb['yellow']
# )

# grnd_mesh = get_ground(grnd_size=10, offset=0.0)
# xmean, ymean, _ = obstacles_dict[receptacle_name][0].mean(0)
# grnd_mesh.v[:, 0] += xmean
# grnd_mesh.v[:, 1] += ymean
# mp_viewer.add_mesh(grnd_mesh.v, grnd_mesh.f, name_to_rgb['brown'])

mp_viewer.add_mesh(body_vertices, sbj_m.faces, name_to_rgb['green'])

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4612175…

1